In [1]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Train JAX/Flax model locally and use `jax2tf` to convert to SavedModel

In [2]:
import logging
import os

import tensorflow as tf
import tensorflow_datasets as tfds
from absl import flags
from jax.experimental.jax2tf.examples import mnist_lib
from jax.experimental.jax2tf.examples import saved_model_lib

In [3]:
TRAIN_BATCH_SIZE = 128
TEST_BATCH_SIZE = 16
SERVING_BATCH_SIZE = 3

NUM_EPOCHS = 2

PROJECT_ID = !(gcloud config get-value project)
PROJECT_ID = PROJECT_ID[0]

REGION = "us-central1"

BUCKET_NAME = PROJECT_ID
# Use a regional bucket in the above region you have rights to.
# Create if needed:
# !gsutil mb -l $REGION gs://$BUCKET_NAME

BASE_OUTPUT_DIR = f"gs://{BUCKET_NAME}"
MODEL_NAME = "jax_model_local"
MODEL_VERSION = 1

# Block TF from the GPU to let JAX use it all
tf.config.set_visible_devices([], 'GPU')

logger = logging.getLogger()

# need to initialize flags somehow to avoid errors in load_mnist
flags.FLAGS([''])

flax_mnist = mnist_lib.FlaxMNIST()

In [4]:
train_ds = mnist_lib.load_mnist(tfds.Split.TRAIN, TRAIN_BATCH_SIZE)
test_ds = mnist_lib.load_mnist(tfds.Split.TEST, TEST_BATCH_SIZE)

In [5]:
logger_level = logger.level
logger.setLevel(logging.INFO)
predict_fn, params = flax_mnist.train(
    train_ds=train_ds,
    test_ds=test_ds,
    num_epochs=NUM_EPOCHS,
)
logger.setLevel(logger_level)

INFO:absl:Starting the local TPU driver.
INFO:absl:Unable to initialize backend 'tpu_driver': Not found: Unable to find driver in registry given worker: local://
INFO:absl:Unable to initialize backend 'tpu': Invalid argument: TpuPlatform is not available.
INFO:root:mnist_flax: Epoch 0 in 5.24 sec
INFO:root:mnist_flax: Training set accuracy 88.50%
INFO:root:mnist_flax: Test set accuracy 88.98%
INFO:root:mnist_flax: Epoch 1 in 1.09 sec
INFO:root:mnist_flax: Training set accuracy 90.81%
INFO:root:mnist_flax: Test set accuracy 91.22%


In [6]:
# Batch-polymorphic SavedModel
input_signatures = [
    tf.TensorSpec((None,) + mnist_lib.input_shape, tf.float32),
]
polymorphic_shapes = "(batch, ...)"

In [7]:
saved_model_lib.convert_and_save_model(
    jax_fn=predict_fn,
    params=params,
    model_dir=os.path.join(
        BASE_OUTPUT_DIR, "model", MODEL_NAME, str(MODEL_VERSION)
    ),
    input_signatures=input_signatures,
    polymorphic_shapes=polymorphic_shapes,
)

INFO:tensorflow:Assets written to: gs://dsparing-sandbox/model/jax_model_local/1/assets


INFO:tensorflow:Assets written to: gs://dsparing-sandbox/model/jax_model_local/1/assets


## Test Prediction with both predict_fn and savedmodel

In [8]:
images_to_predict, _ = next(
    iter(mnist_lib.load_mnist(tfds.Split.TEST, batch_size=SERVING_BATCH_SIZE))
)

In [9]:
predict_fn(params, images_to_predict)

DeviceArray([[ -9.413428  ,  -9.9401655 , -11.998154  ,  -7.7857533 ,
               -3.9688947 ,  -6.5135517 , -12.03955   ,  -0.42528656,
               -6.579282  ,  -1.1267226 ],
             [ -7.438002  , -14.05672   ,  -0.02170406, -12.321133  ,
              -10.795774  , -12.085107  ,  -3.9711492 , -14.134722  ,
               -6.2216706 , -11.326763  ],
             [ -8.40206   , -15.618109  , -11.697708  ,  -8.362864  ,
               -9.140024  ,  -8.687246  , -14.885334  ,  -0.12314819,
               -7.983146  ,  -2.164707  ]], dtype=float32)

In [10]:
!gsutil ls -l $BASE_OUTPUT_DIR/model/$MODEL_NAME/$MODEL_VERSION

         0  2021-06-29T20:39:31Z  gs://dsparing-sandbox/model/jax_model_local/1/
     59519  2021-07-01T14:30:08Z  gs://dsparing-sandbox/model/jax_model_local/1/saved_model.pb
                                 gs://dsparing-sandbox/model/jax_model_local/1/assets/
                                 gs://dsparing-sandbox/model/jax_model_local/1/variables/
TOTAL: 2 objects, 59519 bytes (58.12 KiB)


In [11]:
!saved_model_cli show \
    --dir $BASE_OUTPUT_DIR/model/$MODEL_NAME/$MODEL_VERSION \
    --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['inputs'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 28, 28, 1)
        name: serving_default_inputs:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['output_0'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 10)
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict

Defined Functions:
  Function Name: '__call__'
    Option #1
      Callable with:
        Argument #1
          inputs: TensorSpec(

In [12]:
loaded_model = tf.saved_model.load(
    os.path.join(BASE_OUTPUT_DIR, "model", MODEL_NAME, str(MODEL_VERSION))
)
loaded_model.signatures["serving_default"](images_to_predict)

{'output_0': <tf.Tensor: shape=(3, 10), dtype=float32, numpy=
 array([[ -9.413428  ,  -9.940166  , -11.998155  ,  -7.785753  ,
          -3.9688945 ,  -6.5135503 , -12.039551  ,  -0.4252868 ,
          -6.579281  ,  -1.1267219 ],
        [ -7.438001  , -14.05672   ,  -0.02170417, -12.321133  ,
         -10.795776  , -12.085108  ,  -3.97115   , -14.134721  ,
          -6.2216706 , -11.326763  ],
        [ -8.40206   , -15.618111  , -11.697708  ,  -8.362864  ,
          -9.140025  ,  -8.687246  , -14.885333  ,  -0.12314788,
          -7.9831457 ,  -2.1647086 ]], dtype=float32)>}